# Introduction to CUDA C++

CUDA is a paralell computing platform and programming mdoel from Nvidia. It lets you use C++ to develop high performance algorithms accelerated by paralell threads running on GPUs. This tutorial I followed is based on the Nvidia DLI: [An Even Easier Introduction to CUDA](https://learn.nvidia.com/courses/course?course_id=course-v1:DLI+T-AC-01+V1&unit=block-v1:DLI+T-AC-01+V1+type@vertical+block@e5689ab035a445868125e1b27a258f99)

The below cell allows consistency with Colab's command `%%shell` for running C++ scripts

In [14]:
from IPython.core.magic import register_cell_magic

@register_cell_magic
def shell(line, cell):
    from IPython import get_ipython
    get_ipython().run_cell_magic("shell", line, cell)

The code below adds the elements of two arrays with a million elements each

In [15]:
%%writefile add.cpp

#include <iostream>
#include <math.h>

// function to add the elements of two arrays
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
      y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20; // 1M elements

  float *x = new float[N];
  float *y = new float[N];

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the CPU
  add(N, x, y);

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  delete [] x;
  delete [] y;

  return 0;
}

Overwriting add.cpp


The code below compiles the program and runs it. As you will see, there is no error in the summation.

In [17]:
%%shell
g++ add.cpp -o add
./add

RecursionError: maximum recursion depth exceeded while calling a Python object

Now, to get it running computation in paralell on the many cores of the GPU, first you must turn the `add` function into a function that the GPU can run, called a kernel in CUDA. You do this by adding the `__global__` specifier to the function that tells the CUDA C++ compiler that this is a function that runs on the GPU and can be called from the CPU code. Code running on the GPU is often referred to as *device code* and the code that runs of CPU is *host code*

```cpp
// CUDA Kernel function to add the elements of two arrays on the GPU
__global__
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
      y[i] = x[i] + y[i];
}
```

## Memory Allocation in CUDA

To compute on the GPU, you need to allocate memory accessible by the GPU. CUDA uses *unified memory*, meaning it has a shared pool of high-bandwidth memory accesible by both CPU and GPU integrated onto a single chip, eliminating data copying between the two memories. 

To allocate data to unified memory, you must call `cudaMallocManaged()`, which returns a pointer that you can access from host code or device code. To free the data again, call `cudaFree()`. Replace the code above that calls `new` with calls to `cudaMallocManaged()` and `delete` as `cudaFree`

```cpp  
// Allocate Unified Memory -- accessible from CPU or GPU
  float *x, *y;
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  ...

  // Free memory
  cudaFree(x);
  cudaFree(y);
```

Finally, you must launch the `add()` kernel, invoking it on the GPU. CUDA kernel launches are denoted using `<<< >>>`, just trun the `add` before the parameter list. You also must make the CPU wait until the kernel is done before it accesses the results because the CUDA kernel launches don't block the CPU thread being called. To do this, call `cudaDeviceSynchronize()` before doing the final error check.

```cpp
add<<<1, 1>>>(N, x, y);
```

In [ ]:
%%writefile add.cu

#include <iostream>
#include <math.h>
// Kernel function to add the elements of two arrays
__global__
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
    y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<20
 ;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  add<<<1, 1>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);
  
  return 0;
}